In [1]:
from source import *
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import warnings; warnings.filterwarnings('ignore')

# Data Science Case Study: Supplier Audit Results Prediction

by: Juan Betancourt

juan.beta98@gmail.com

# Contents

1. Brief Case Study Description

2. Data Exploration and Assumptions Formulation

3. Preprocessing the Data

4. Predictor Selection

5. Best Model Selection

6. Prediction for Audit Results

7. Business Recomendations

8. Model Maintenance

# 1.Brief Case Study Description


<br><div style="text-align: justify"> 
Managing suppliers is a critical aspect of the operational efficiency of any manufacturing company. Issues such as irregular lead times, backlogs, and incorrect orders can significantly disrupt manufacturing schedules, increase costs, and ultimately impact the service level provided to customers. In the context of this case study, our focus is on predicting the outcome of audits for a specific supplier and derivative combination.
</div> 

<br><div style="text-align: justify"> 
The process involves three stages: T1, T2, and T3. The goal is to forecast the most recent qualification status for a given supplier and derivative in their final audit stage. To achieve this objective, we have access to two datasets. The first dataset comprises essential information regarding the audit history of suppliers and the derivatives under their management. The second dataset provides diverse metrics related to the performance of suppliers.
</div> 

# 2.Data Exploration

## 2.1 Initial Exploration

The initial stage of data exploration involves obtaining a comprehensive overview of the datasets. This includes understanding the structure, content, and key characteristics of the available data.

### 2.1.1 _AuditHistory_ dataset

In [2]:
AuditHistory.head()

,SupplierId,RecentMonth,RecentTimeStamp,DerivativeName,DerivativeRegion,ProductionLine,T1Color,T2Color,T3Color,Result
0,f763999191217e9945b26d312d0902d5,2022-06,2022-06-30T10:01:20Z,3c4d48ad7d892bfe96a6df0fabe6deaa,Asia,LU,Green,Green,-,Green
1,377c62276b7dd74e406ac5de3c872110,2022-09,2022-09-28T07:00:46Z,2dc5fe1b4f8895bb1096f6470b7d8550,Europe,LG,Green,Green,Green,Green
2,20beb9edfc8536f19109a15b5ce1a845,2022-09,2022-09-28T07:00:46Z,9c491ff90cbfc2882aa1116944658240,Europe,LC,Green,Green,Green,Green
3,d0ae2fdd6681f6f99469af4f1ede3928,2022-10,2022-10-31T11:01:25Z,3724c242924ad7b7871634355d6ee63e,USA,LC,Green,Green,Green,Green
4,4668950356774590b25d92473b8756d9,2022-02,2022-02-28T11:01:17Z,99904aba55dd386296e5925bac70abe0,Europe,LU,Green,Red,-,Red


In [3]:
AuditHistory.describe()

,SupplierId,RecentMonth,RecentTimeStamp,DerivativeName,DerivativeRegion,ProductionLine,T1Color,T2Color,T3Color,Result
count,7690,7690,7690,7690,7547,7690,7690,7690,7690,7690
unique,818,9,9,75,5,4,3,4,4,3
top,4600f0a901054216cf0eecf2999f7eb2,2022-10,2022-10-31T11:01:25Z,9c491ff90cbfc2882aa1116944658240,Europe,LC,Green,Green,Green,Green
freq,36,989,989,760,3643,2737,7417,5499,4178,6797


In [4]:
AuditHistory.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7690 entries, 0 to 7689
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   SupplierId        7690 non-null   object
 1   RecentMonth       7690 non-null   object
 2   RecentTimeStamp   7690 non-null   object
 3   DerivativeName    7690 non-null   object
 4   DerivativeRegion  7547 non-null   object
 5   ProductionLine    7690 non-null   object
 6   T1Color           7690 non-null   object
 7   T2Color           7690 non-null   object
 8   T3Color           7690 non-null   object
 9   Result            7690 non-null   object
dtypes: object(10)
memory usage: 600.9+ KB


This dataset has 10 columns and 7690 entries. As there are various categorical features, here are some details:

- The dataset covers a time interval of 9 different months.
- A total of 818 suppliers, 75 derivatives, five (5) regions, and four (4) production lines are registered.

The only column with missing values is the _derivative region_ with 143 missing values.

In [5]:
plot_audit_colors(AuditHistory,'f763999191217e9945b26d312d0902d5','3c4d48ad7d892bfe96a6df0fabe6deaa')

AttributeError: 'DataFrame' object has no attribute 'sort_intex'

### 2.1.2 _SupplierPerformance_ dataset

In [ ]:
SupplierPerformance.head()

In [ ]:
SupplierPerformance.describe(include='all')

In [ ]:
SupplierPerformance.info()

This dataset comprises 8 columns and 5823 entries. Here, most features, except for _BadSupplierIndicator_, consist of numeric data. Here are some details:

- Suppliers exhibit a significantly higher number of backlogs than wrong deliveries.

<br><div style="text-align: justify"> 
Two columns have missing data: *12_Month_LPKM* with 325 missing values and _BadSupplierIndicator_ with 5491 missing values. Upon closer inspection of the latter, we find that this column is encoded with 'bad' for suppliers with previous poor performance and nothing for all others. Therefore, one-hot encoding this column with 1 for poor performance and 0 for all others will resolve this issue.
</div> 

<br><div style="text-align: justify"> 
Another important detail is that this table has 5823 entries, each corresponding to a different supplier. In the _AuditHistory_ dataset, there are only 818 suppliers, indicating that most of the suppliers in this dataset lack audit data. These suppliers will be discarded, as there is not sufficient data to incorporate them into the model.
</div> 

In [ ]:
SupplierPerformance['BadSupplierIndicator']=SupplierPerformance['BadSupplierIndicator'].fillna(0).map({'bad':1,0:0})
SupplierPerformance = SupplierPerformance[SupplierPerformance.index.isin(Suppliers)]

## 2.2 Distribution Analysis

The next stage of dataset exploration involves conducting a distribution analysis. This analysis aims to provide insights into the distribution patterns of both numeric and categorical features.

### 2.2.1 _AuditHistory_ dataset

In [ ]:
cols = list(AuditHistory.columns)[1:]
col_name = cols[5]
plot_categorical_distribution(col_name)

Some relevant insights of the features:

- The number of observations exhibits a growing trend from February 2022 to October 2022, indicating an increasing volume over time.
- The majority of derivatives are sourced from Europe, with South Africa contributing only a minimal number of units.
- Among production lines, LC stands out with the highest number of observations, exceeding 2500, while LG has a comparatively lower count, with around 1250 observations.
- In Phase T1, the majority of observations have green qualifications, with very few instances of yellow and red qualifications. In Phase T2, there is also a predominance of green qualifications, but unlike T1, T2 has a notable number of empty observations, along with very few yellow and red qualifications. Moving to Phase T3, the number of green observations is lower (around 4200), accompanied by a significant number of empty observations (approximately 3100). The occurrences of yellow and red qualifications are still relatively higher compared to T1 and T2.

### 2.2.2 _SupplierPerformance_ dataset

In [ ]:
cols = list(SupplierPerformance.columns)
col_name = cols[7]
plot_numerical_distribution(col_name)

Some relevant insights of the features:

- The difference between the amount of wrong deliveries and the backlogs is displayed with the histogram. 


## 2.3 Correlation

### 2.3.1 _AuditHistory_ dataset

### 2.3.2 _SupplierPerformance_ dataset

## 2.4 Box Plots

### 2.4.1 _AuditHistory_ dataset

### 2.4.2 _SupplierPerformance_ dataset

## 2.5 Missing Values

### 2.5.1 _AuditHistory_ dataset

### 2.5.2 _SupplierPerformance_ dataset

## 2.6 Outlier Detection

### 2.6.1 _AuditHistory_ dataset

### 2.6.2 _SupplierPerformance_ dataset

## 2.7 Assumptions

1. The _AuditHistory_ and _SupplierPerformance_ datasets are chronologically linked. This implies that the three, six, and 12-month markers in the latter dataset correspond to the respective preceding three, six, and 12 months relative to the latest audit date.

# 3.Preprocessing the Data

The initial data processing steps involve assigning numerical values to categorical features, such as _DerivativeRegion_ and _ProductionLine_. Additionally, the _RecentTimeStamp_ column will be removed, as _RecentMonth_ provides a more concise representation of the time observation.

In [ ]:
AuditHistory,encoding = code_categorical_columns(AuditHistory,['DerivativeRegion','ProductionLine','T1Color','T2Color','T3Color','Result'])
AuditHistory.drop(labels='RecentTimeStamp',axis=1,inplace=True)
AuditHistory.head()

My first approach is to build a Machine Learning algorithm for each stage (T1, T2 and T3). Each of the algorithms should be able to predict the qualification for that stage. Having the prediction of the three algorithms, the result can be obtained as the qualification of the latest audit stage.



Therefore, the idea is to build a dataset that has observations with information about both suppliers and derivatives. Another relevant aspect to consider is that the _AuditHistory_ dataset contains time series as it is indexed in time. Then, particular observations of this table must be placed correctly in time. 

Then, this model should work for any of the 5823 suppliers in the _SupplierPerformance_ dataset. Therefore, feature engineering will be critical to gather all the relevant information. For any supplier - Derivative combination, there is _DerivativeRegion_ and _ProductionLine_ information.  

To sumarize, the predictors of the result of an audit for a given supplier and a given derivative are:

1. DerivativeRegion
2. ProductionLine
3. Latest T1 qualification
4. Time since last T1 change
5. Latest T2 qualification
6. Time since last T2 change
7. Latest T3 qualification
8. Time since last T2 change
9. Time since last audit
10. Wrong orders in the last 3 months
11. 12 Month LPKM
12. BadSupplierIndicator

# 4.Predictor Selection

# 5.Best Model Selection

# 6.Prediction for Audit Results

# 7.Business Recomendations

# 8.Model Maintenance

### 9. Additional Ideas

Throughout the course of this project, several ideas emerged that, unfortunately, I couldn't fully explore due to time constraints. Nevertheless, I would like to present them:

1. Supplier Clustering: Given the abundance of suppliers without audit records, their information was excluded from the primary analysis. However, there is an opportunity for further investigation. For instance, employing a clustering algorithm could group these suppliers, allowing the extraction of pertinent features. These features could then be utilized as input for a machine learning classification algorithm.